In [1]:
import sys
import tensorflow as tf
sys.path.append('../../generative-adversarial-nets/')
from ganetwork import GAN, CGAN
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer

In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [3]:
discriminator_layers=[(4, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(6, None), (5, tf.nn.tanh), (4, None)]
gan = GAN(discriminator_layers, generator_layers)
gan.train(X, nb_epoch=50, batch_size=5, discriminator_steps=3)

Epoch: 0, discriminator loss: 1.4024484157562256, generator loss: 0.5729442834854126
Epoch: 1, discriminator loss: 1.3899049758911133, generator loss: 0.6406246423721313
Epoch: 2, discriminator loss: 1.3900309801101685, generator loss: 0.6707156896591187
Epoch: 3, discriminator loss: 1.397815227508545, generator loss: 0.6935044527053833
Epoch: 4, discriminator loss: 1.4008744955062866, generator loss: 0.6928763389587402
Epoch: 5, discriminator loss: 1.4091665744781494, generator loss: 0.6804187297821045
Epoch: 6, discriminator loss: 1.4003655910491943, generator loss: 0.6846488118171692
Epoch: 7, discriminator loss: 1.3969646692276, generator loss: 0.6870303750038147
Epoch: 8, discriminator loss: 1.3992950916290283, generator loss: 0.6859229207038879
Epoch: 9, discriminator loss: 1.3931716680526733, generator loss: 0.6898093223571777
Epoch: 10, discriminator loss: 1.3834271430969238, generator loss: 0.6859794855117798
Epoch: 11, discriminator loss: 1.3806657791137695, generator loss: 0

In [4]:
discriminator_layers=[(7, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(9, None), (5, tf.nn.tanh), (4, None)]
cgan = CGAN(discriminator_layers, generator_layers)
cgan.train(X, y, nb_epoch=50, batch_size=5)

Epoch: 0, discriminator loss: 1.904671311378479, generator loss: 0.20133432745933533
Epoch: 1, discriminator loss: 1.8282910585403442, generator loss: 0.2253890037536621
Epoch: 2, discriminator loss: 1.7215604782104492, generator loss: 0.2670222818851471
Epoch: 3, discriminator loss: 1.6061646938323975, generator loss: 0.32706597447395325
Epoch: 4, discriminator loss: 1.5293996334075928, generator loss: 0.3813930153846741
Epoch: 5, discriminator loss: 1.4856481552124023, generator loss: 0.424365371465683
Epoch: 6, discriminator loss: 1.462916612625122, generator loss: 0.4516311585903168
Epoch: 7, discriminator loss: 1.4430131912231445, generator loss: 0.48590660095214844
Epoch: 8, discriminator loss: 1.4309736490249634, generator loss: 0.5084179639816284
Epoch: 9, discriminator loss: 1.4216632843017578, generator loss: 0.5383267998695374
Epoch: 10, discriminator loss: 1.4259769916534424, generator loss: 0.543580174446106
Epoch: 11, discriminator loss: 1.406829833984375, generator loss: